# Add sticky notes randomly in a shape

This notebook demonstrates how to add new sticky notes inside a shape in a random pattern.

There is no sample mural to start this notebook.  Just make a new mural yourself with one large rectangle in it.

When you run this notebook, your mural will look something like the following image:

<img src="https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/images/sample-07_add-sticky-notes.gif" width="50%" title="Image of a mural" />

## Step 1: Collect the mural ID

You can find the mural ID in the url of a mural.

Mural urls look something like this:

```
https://app.mural.co/t/<workspace>/m/<workspace>/<id>/...
```

What you need to pass to the MURAL API is just after the `/m/`: the \<workspace> and the \<id>.  And you need to join then with a period.

For example, if you have a mural with this url:

```
https://app.mural.co/t/teamideas1234/m/teamideas1234/1234567890123/...
```

Then, the mural ID is: `teamideas1234.1234567890123`

In [1]:
g_mural_id = ""

## Step 2: Collect your OAuth token

In [80]:
g_auth_token = ""

## Step 3: Read the rectangle shape from the mural

In [81]:
import requests
import json

def listWidgets( mural_id, auth_token ):
    # https://developers.mural.co/public/reference/getmuralwidgets
    url = "https://app.mural.co/api/public/v1/murals/" + mural_id + "/widgets"
    headers = { "Accept": "application/json", "Authorization": "Bearer " + auth_token }
    response = requests.request( "GET", url, headers = headers )
    response_json = json.loads( response.text )
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( msg )
        return None
    if "value" not in response_json:
        print( "No value returned" )
        return None
    return response_json["value"]

In [82]:
widgets_arr = listWidgets( g_mural_id, g_auth_token )

In [83]:
rectangle = { "id"     : widgets_arr[0]["id"],
              "x"      : widgets_arr[0]["x"],
              "y"      : widgets_arr[0]["y"],
              "width"  : widgets_arr[0]["width"],
              "height" : widgets_arr[0]["height"] }
print( json.dumps( rectangle, indent = 3 ) )

{
   "id": "0-1651775781700",
   "x": 169.43,
   "y": 151.37,
   "width": 1712.8376623376619,
   "height": 849.0389610389609
}


## Step 4: Add stickies inside the rectangle randomly

### 4.1 Coordinates
Make a list of random x, y coordinates inside the rectangle.

In [66]:
import numpy as np
import random

def makeRandomCoords( rectangle, sticky_width, sticky_height ):
    x_start = round( rectangle["x"] + ( 0.5 * sticky_width ) )
    x_end   = round( rectangle["x"] + rectangle["width"] - ( 1.5 * sticky_width ) )
    y_start = round( rectangle["y"] + ( 0.5 * sticky_height ) )
    y_end   = round( rectangle["y"] + rectangle["height"] - ( 1.5 * sticky_height ) )
    x_list = np.arange( x_start, x_end, round( 0.6 * sticky_width ) ).tolist()
    x_list += x_list
    random.shuffle( x_list )
    random.shuffle( x_list )
    y_list = np.arange( y_start, y_end, round( 0.6 * sticky_width ) ).tolist()
    y_list += y_list
    random.shuffle( y_list )
    random.shuffle( y_list )
    x_length = len( x_list )
    y_length = len( y_list )
    list_length = x_length if ( x_length < y_length ) else y_length
    coords = []
    for i in range( list_length ):
        x = x_list[i] + 10 * random.random()
        y = y_list[i] + 10 * random.random()
        coords.append( [ x, y ] )
    return coords

### 4.2 Colors
Make a random list of nice background colors for the stickies.

In [74]:
g_colors = {
    "#b68d40FF" : "tan",
    "#f4ebd0FF" : "cream",
    "#122620FF" : "charcoal",
    "#b99095FF" : "mauve",
    "#fcb5acFF" : "salmon",
    "#b5e5cfFF" : "mint",
    "#3d5b59FF" : "teal green",
    "#05445eFF" : "navy blue",
    "#189ab4FF" : "blue grotto",
    "#d4f1f4FF" : "baby blue"
}

def randomColors():
    color_index_list = np.arange( 0, len( g_colors.keys() ), 1 ).tolist()
    color_index_list += color_index_list
    random.shuffle( color_index_list )
    random_colors = []
    for color_index in color_index_list:
        random_colors.append( list( g_colors.keys() )[ color_index ] )
    return random_colors

### 4.3 Reuse the is-it-overlapping function
We'll need to reuse this function from earlier samples so we can avoid piling sticky notes right on top of each other.

In [27]:
def stickyInsideOverlapsRectangle( sticky, rectangle ):
    #
    left_edge_overlaps = False
    if ( sticky["x"] >= rectangle["x"] ) and \
       ( sticky["x"] <= ( rectangle["x"] + rectangle["width"] ) ):
        left_edge_overlaps = True
    #
    right_edge_overlaps = False
    if ( ( sticky["x"] + sticky["width"] ) >= rectangle["x"] ) and \
       ( ( sticky["x"] + sticky["width"] ) <= ( rectangle["x"] + rectangle["width"] ) ):
        right_edge_overlaps = True
    #
    top_edge_overlaps = False
    if ( sticky["y"] >= rectangle["y"] ) and \
       ( sticky["y"] <= ( rectangle["y"] + rectangle["height"] ) ):
        top_edge_overlaps = True
    #
    bottom_edge_overlaps = False
    if ( ( sticky["y"] + sticky["height"] ) >= rectangle["y"] ) and \
       ( ( sticky["y"] + sticky["height"] ) <= ( rectangle["y"] + rectangle["height"] ) ):
        bottom_edge_overlaps = True
    
    if left_edge_overlaps and right_edge_overlaps and top_edge_overlaps and bottom_edge_overlaps:
        return "inside"
    
    if ( left_edge_overlaps or right_edge_overlaps ) and ( top_edge_overlaps or bottom_edge_overlaps ):
        return "overlaps"
    
    return "outside"

def overlaps( x, y, sticky_width, sticky_height, stickies_arr ):
    sticky_in = { "x" : x, "y" : y, "width" : sticky_width, "height" : sticky_height }
    for existing_sticky in stickies_arr:
        position = stickyInsideOverlapsRectangle( sticky_in, existing_sticky )
        if ( "overlaps" == position ) or ( "inside" == position ):
            return True
    return False

### 4.4 MURAL API
Once we know where we want to put a sticky note, we need to use the MURAL API to put it there.

In [31]:
def addStickyToMural( mural_id, auth_token, sticky_data ):
    # https://developers.mural.co/public/reference/createstickynote
    url = "https://app.mural.co/api/public/v1/murals/" + mural_id + "/widgets/sticky-note"
    headers = { "Content-Type" : "application/json", "Accept" : "vnd.mural.preview", "Authorization" : "Bearer " + auth_token }
    data = json.dumps( sticky_data )
    response = requests.request( "POST", url, headers=headers, data=data )
    response_json = json.loads( response.text )
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( msg )

### 4.5 Pull it all together
Here's the routine using all of the above components:

In [71]:
def placeAllStickies( mural_id, auth_token, rectangle, sticky_width, sticky_height, text_arr ):
    coords = makeRandomCoords( rectangle, sticky_width, sticky_height )
    random_colors = randomColors()
    stickies_arr = []
    for txt in text_arr:
        bg_color = random_colors.pop()
        print( "[ " + g_colors[ bg_color ].ljust(10) + " ]  " + txt )
        x, y = coords.pop()
        count = 0
        while ( count <= 3 ) and overlaps( x, y, sticky_width, sticky_height, stickies_arr ):
            x, y = coords.pop()
            count += 1
        sticky = { "x"      : x, 
                   "y"      : y, 
                   "width"  : sticky_width, 
                   "height" : sticky_height, 
                   "shape"  : "rectangle", 
                   "style"  : { "backgroundColor" : bg_color }, 
                   "text"   : txt }
        addStickyToMural( mural_id, auth_token, sticky )
        stickies_arr.append( sticky )

In [72]:
g_text_arr = [
    "Here's a sticky note!",
    "I love posting stickies",
    "comment comment comment comment",
    "Look at all these comments",
    "Yet another sticky",
    "You guessed it.. another sticky note",
    "Is it getting crowded in this rectangle yet?",
    "How's the weather where you are?",
    "Apples are quite nice",
    "Don't forget an umbrella!"
]

In [84]:
import time

time.sleep(5)

# Quick!  After running this cell, switch to your browser 
# tab where the mural is to see the stickies get added
# ...

g_width = 150
g_height = 150

placeAllStickies( g_mural_id, g_auth_token, rectangle, g_width, g_height, g_text_arr )

[ baby blue  ]  Here's a sticky note!
[ navy blue  ]  I love posting stickies
[ charcoal   ]  comment comment comment comment
[ navy blue  ]  Look at all these comments
[ salmon     ]  Yet another sticky
[ teal green ]  You guessed it.. another sticky note
[ tan        ]  Is it getting crowded in this rectangle yet?
[ mauve      ]  How's the weather where you are?
[ mint       ]  Apples are quite nice
[ tan        ]  Don't forget an umbrella!


Something like this:

<img src="https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/images/sample-07_add-sticky-notes.gif" width="50%" title="Image of a mural" />